# Vowel Data Collection

This notebook includes all code involved in collecting formant data for vowels, working with the alignment data collected from the phoneme alignment notebook.

## Setup

In [ ]:
import numpy as np

import parselmouth
import matplotlib.pyplot as plt

from tqdm import tqdm
import json

In [ ]:
# Prepare texts
TEXTS = [
    "Petey's mom got a CD record of 'Everlong' by Foo Fighters.",
    "Jude said goodbye to his pet goose when he moved to Tuscon, Arizona.",
    "I went to watch a movie instead, and I met my friend there.",
    "My dog thought it was odd that he never got to eat human food.",
    "Who bought my seat at the 3D movie theater?",
    "My job schedule is full – can Bob do Tuesday at the beach instead?"
]

# Prepare audio titles
AUDIOS_1 = ["petey1", "jude1", "movie1", "dog1", "who1", "job1"]
AUDIOS_2 = ["petey2", "jude2", "movie2", "dog2", "who2", "job2"]
AUDIOS_3 = ["petey3", "jude3", "movie3", "dog3", "who3", "job3"]
AUDIOS_4 = ["petey4", "jude4", "movie4", "dog4", "who4", "job4"]

# Prepare audio filenames
FILES_1 = [f"audios/{audio}.wav" for audio in AUDIOS_1]
FILES_2 = [f"audios/{audio}.wav" for audio in AUDIOS_2]
FILES_3 = [f"audios/{audio}.wav" for audio in AUDIOS_3]
FILES_4 = [f"audios/{audio}.wav" for audio in AUDIOS_4]

# Prepare testing conditions
CONDITIONS = [FILES_1, FILES_2, FILES_3, FILES_4]

# Prepare vowels
VOWELS = ["i", "e", "a", "u"]

# Load in alignments
with open('alignments.json') as json_file:
    ALIGNMENTS = json.load(json_file) 

## Get the formants of vowels from an audio file

In [ ]:
# get F1/F2 data from a wavfile
def get_formant_data(file):
    snd = parselmouth.Sound(file)
    formant = snd.to_formant_burg()
    time_values = formant.xs()
    f1_values = []
    f2_values = []
    for t in time_values:
        f1_values.append(formant.get_value_at_time(1, t))
        f2_values.append(formant.get_value_at_time(2, t))
    f1_values = np.array(f1_values)
    f2_values = np.array(f2_values)
    return np.vstack((time_values, f1_values, f2_values))

# from an alignment data chunk, get the starts and ends of each vowel
def get_vowel_annotations(alignment):
    i_annotations = []
    e_annotations = []
    a_annotations = []
    u_annotations = []
    for start, end, sound in alignment:
        if sound == "IY":
            i_annotations.append((start, end))
        elif sound == "EH":
            e_annotations.append((start, end))
        elif sound == "AA":
            a_annotations.append((start, end))
        elif sound == "UW":
            u_annotations.append((start, end))
    return i_annotations, e_annotations, a_annotations, u_annotations

# Get the formants of a vowel, given the formant data and annotations
def get_vowel_data(formant_data, vowel_annotations):
    vowel_f1 = np.array([])
    vowel_f2 = np.array([])
    for start, end in vowel_annotations:
        time = formant_data[0]
        mask = (time > start) & (time < end)
        vowel_f1 = np.append(vowel_f1, formant_data[1, mask])
        vowel_f2 = np.append(vowel_f2, formant_data[2, mask])
    return np.vstack((vowel_f1, vowel_f2))

## Collect data for one sentence in one condition

In [ ]:
def get_vowel_data_one_condition(condition, sentence):
    # Find the text and file
    c_id = condition - 1
    s_id = sentence - 1
    
    text = TEXTS[s_id]
    files = CONDITIONS_F[c_id]
    file = files[s_id]
    
    # Find the alignment
    label = f"c{c_id}s{s_id}"
    alignment = ALIGNMENTS[label]
    
    # Get the vowel formant data
    all_vowel_annotations = get_vowel_annotations(alignment)
    formant_data = get_formant_data(file)
    vowel_formants = [get_vowel_data(formant_data, v) 
                      for v in all_vowel_annotations]

    # Package this data with vowel labels
    vowel_data = {label: formants 
                  for label, formants in zip(VOWELS, all_vowel_formants)}

    return vowel_data

## Main function: get all vowel data

In [ ]:
def get_all_vowel_data():
    all_vowel_data = {}
    for c_id in tqdm(range(4)):
        for s_id in range(6):
            label = f"c{c_id}s{s_id}"
            data = analyze_vowels(c_id + 1, s_id + 1)
            all_vowel_data[label] = data
    return all_vowel_data

all_vowel_data = get_all_vowel_data()

## Save vowel data

In [ ]:
# Numpy arrays are not serializable, so convert them to lists.
def serialize_data(data):
    def convert(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
    return json.dump(data, default=convert)

with open("vowel_data.json", "w") as outfile: 
    json.dump(sdf, outfile)